# Metadata

**L1 Taxonomy** - Background Processes

**L2 Taxonomy** - Job Scheduling

**Subtopic** - Persisting the state of scheduled jobs so they can resume after a restart

**Use Case** - Develop a Python script that uses the 'schedule' package to create a job scheduler. The script should be able to schedule multiple jobs and persist their state to a local file. After a restart, the script should read the state from the file and resume the scheduled jobs.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
schedule==1.2.2
```


# Prompt
Problem Statement:
- Develop a Python script that uses the 'schedule' package to create a job scheduler.
- The script should be able to schedule multiple jobs and persist their state to a local file.
- After a restart, the script should read the state from the file and resume the scheduled jobs.

Input Format:
- A list of job definitions, where each job has:
  - A unique job ID (string)
  - An interval (integer) specifying how often to run the job
  - A time unit (string): one of "seconds", "minutes", "hours", "days"
  - An action name (string) corresponding to a predefined function in the script
  - A list of arguments (list of strings or values) to pass to the action

Input Constraints:
- 1 <= number of jobs <= 100
- Job ID must be a non-empty string of alphanumeric characters (max length: 64)
- Interval must be a positive integer (1 <= interval <= 1000000)
- Time unit must be one of: "seconds", "minutes", "hours", "days"
- Action name must match a predefined function in the script
- Arguments list must contain only JSON-serializable values
- Note: The 'schedule' package uses polling and is not suitable for precise second-level accuracy. Jobs may be delayed by up to 1 second.

Output Format:
- The scheduler prints a message each time a job is executed.
- The message format is:
  [<timestamp>] Executed job <job_id>: <action> <args>
- On shutdown, the scheduler saves all active job definitions to a local file named "jobs_state.json".
- On startup, it loads "jobs_state.json" (if it exists) and resumes all scheduled jobs.

Class Definition:
```python
class JobScheduler:
    def __init__(self, state_file: str) -> None
    def add_job(self, job_id: str, interval: int, unit: str, action: str, args: list) -> None
    def start(self) -> None
    def shutdown(self) -> None
    def load_jobs(self) -> None
    def save_jobs(self) -> None
```

Example:
```python
Example Input (jobs.json):
[
  {
    "id": "job1",
    "interval": 5,
    "unit": "seconds",
    "action": "print_message",
    "args": ["Job 1 executed"]
  },
  {
    "id": "job2",
    "interval": 1,
    "unit": "minutes",
    "action": "print_message",
    "args": ["Job 2 running every minute"]
  }
]

Execution Behavior:
- Every 5 seconds, output:
  [2025-08-01T18:00:05.123456] Job 1 executed
- Every 1 minute, output:
  [2025-08-01T18:01:00.654321] Job 2 running every minute

On Shutdown:
- The current jobs are saved to jobs_state.json in the same format as the input.

On Restart:
- The scheduler loads jobs_state.json and resumes running both jobs at their correct intervals.
```

# Requirements
Explicit Requirements:
- Use the 'schedule' package to manage job scheduling.
- Allow multiple jobs to be scheduled with different intervals and actions.
- Persist scheduled jobs to a local file named "jobs_state.json".
- On startup, reload and resume jobs from "jobs_state.json" if it exists.
- Each job must have a unique ID, interval, unit, action, and optional arguments.
- The scheduler must print a message each time a job executes.
- The JobScheduler must run an execution loop that calls schedule.run_pending() at regular intervals (e.g., every second).

Implicit Requirements:
- Job functions (actions) must be predefined and callable by name.
- Scheduled jobs must continue to run in the background until the script is stopped.
- File I/O must be handled safely to prevent data loss or corruption.
- Invalid job definitions (e.g., missing fields or invalid types) should be rejected gracefully.
- On restart, the scheduler resumes job execution based on the current time and defined intervals.
- Missed executions during downtime will not be retroactively triggered.
- The scheduler loads jobs_state.json and reschedules jobs based on the current time. Scheduled jobs resume from now, not from their original timeline.
- The design should support adding new jobs dynamically during runtime.

Solution Expectations:
- The script must define a JobScheduler class that manages job registration, execution, persistence, and recovery.
- The scheduler must use the 'schedule' package to run jobs at specified intervals.
- The script must define and register all supported job actions (e.g., print_message, run_backup).
- The JobScheduler must support adding jobs both at startup (from file) and at runtime (via method call).
- Jobs must be persisted in a JSON file ("jobs_state.json") before shutdown or on update.
- The scheduler must resume from the persisted state on the next run, with no job duplication.
- Job execution must print logs in the expected format with timestamps and job IDs.
- The script should handle common edge cases (e.g., corrupted state file, invalid job definition) gracefully.
- The design should allow easy extension to support new job actions or scheduling features.
- Persisted jobs must retain the same structure as input format unless explicitly extended in future versions.

Edge Cases and Behavior:
- If the state file "jobs_state.json" is missing:
  - The scheduler should start with no jobs and continue running normally.

- If the state file exists but contains invalid JSON:
  - The scheduler should log an error and start with no jobs, without crashing.

- If a job definition is missing required fields (e.g., interval, action):
  - The scheduler should skip that job and log a warning.

- If a job has an unsupported time unit:
  - The scheduler should reject the job and log an appropriate message.

- If the action name does not map to a known function:
  - The job should not be scheduled, and an error should be logged.

- If duplicate job IDs are provided:
  - Only the first job with that ID should be scheduled; others should be ignored with a warning.

- If the script is interrupted (e.g., KeyboardInterrupt):
  - It should gracefully shut down and save the current state.

- If arguments passed to a job are not JSON-serializable:
  - The job should be skipped or raise a validation error before scheduling.

Solution Constraints:
- Only standard Python 3 libraries and the 'schedule' package may be used.
- The solution must be compatible with Python 3.7 or higher.
- Jobs must be persisted in a single local JSON file ("jobs_state.json").
- All job definitions and persisted state must be JSON-serializable.
- The scheduler must support at least 100 concurrent jobs without performance issues.
- No external databases or network calls are allowed.
- The script must not use multithreading or multiprocessing for job scheduling.
- All logs and outputs must be printed to the standard output (console).
- Time units must strictly match those supported by the 'schedule' package.

In [ ]:
# code
"""
A persistent job scheduler using the 'schedule' package.

This module defines a `JobScheduler` class that can schedule and run jobs
at specified intervals, with the ability to save and load job state
to and from a local JSON file.
"""

import json
import logging
import os
import time
from typing import Any, Dict, List

import schedule

logging.basicConfig(
    level=logging.INFO,
    format='[%(asctime)s] %(message)s',
    datefmt='%Y-%m-%dT%H:%M:%S'
)


class JobScheduler:
    """Schedule and manage jobs with state persistence to a local JSON file."""

    def __init__(self, state_file: str) -> None:
        """
        Initialize the scheduler.

        :param state_file: The path to the file for persisting job state.
        """
        self.state_file = state_file
        self.jobs: Dict[str, Dict[str, Any]] = {}
        self.known_actions = {
            'print_message': self._print_message,
        }

    def _print_message(self, *args: Any) -> None:
        """
        Print a message to the console.

        This is a predefined job action.
        """
        message = ' '.join(str(arg) for arg in args)
        logging.info(f"Executed job "
                     f"{schedule.get_jobs()[0].job_id}: {message}")

    def add_job(
        self,
        job_id: str,
        interval: int,
        unit: str,
        action: str,
        args: List[Any]
    ) -> None:
        """
        Add a new job to the scheduler.

        :param job_id: A unique identifier for the job.
        :param interval: The frequency interval.
        :param unit: The time unit (e.g., 'seconds', 'minutes').
        :param action: The name of the action function to call.
        :param args: A list of arguments to pass to the action.
        """
        if job_id in self.jobs:
            logging.warning(
                f"Job ID '{job_id}' already exists. Skipping addition."
            )
            return

        if not self._validate_job(job_id, interval, unit, action, args):
            return

        job_definition = {
            "id": job_id,
            "interval": interval,
            "unit": unit,
            "action": action,
            "args": args
        }
        self.jobs[job_id] = job_definition
        self._schedule_job(job_definition)

    def _validate_job(
        self,
        job_id: str,
        interval: int,
        unit: str,
        action: str,
        args: List[Any]
    ) -> bool:
        """
        Validate a job's definition.

        :return: True if valid, else False.
        """
        if not all([job_id, interval, unit, action]):
            logging.error(
                f"Invalid job definition for job_id '{job_id}': "
                "Missing required fields."
            )
            return False

        if (not isinstance(job_id, str)
                or not job_id.isalnum() or len(job_id) > 64):
            logging.error(
                f"Invalid job ID '{job_id}'. Must be a non-empty "
                "alphanumeric string (max 64 chars)."
            )
            return False

        if not isinstance(interval, int) or not 1 <= interval <= 1000000:
            logging.error(
                f"Invalid interval for job '{job_id}'. Must be a "
                "positive integer (1-1,000,000)."
            )
            return False

        if unit not in ["seconds", "minutes", "hours", "days"]:
            logging.error(
                f"Invalid time unit '{unit}' for job '{job_id}'. "
                "Must be one of 'seconds', 'minutes', 'hours', 'days'."
            )
            return False

        if action not in self.known_actions:
            logging.error(
                f"Unknown action '{action}' for job '{job_id}'. "
                "Action must be predefined."
            )
            return False

        try:
            json.dumps(args)
        except TypeError:
            logging.error(
                f"Arguments for job '{job_id}' are not JSON-serializable."
            )
            return False

        return True

    def _schedule_job(self, job_definition: Dict[str, Any]) -> None:
        """Schedule a single job using the 'schedule' package."""
        job_id = job_definition["id"]
        interval = job_definition["interval"]
        unit = job_definition["unit"]
        action = job_definition["action"]
        args = job_definition.get("args", [])

        action_func = self.known_actions.get(action)
        if not action_func:
            logging.error(
                f"Cannot schedule job '{job_id}'. Action '{action}' "
                "is not defined."
            )
            return

        try:
            job = getattr(schedule.every(interval), unit).do(
                action_func, *args
            )
            job.job_id = job_id
            logging.info(
                f"Scheduled job '{job_id}' to run every {interval} {unit}."
            )
        except Exception as e:
            logging.error(f"Failed to schedule job '{job_id}': {e}")

    def start(self) -> None:
        """
        Start the scheduler's main loop.

        This method loads any existing jobs from the state file before
        starting the execution loop.
        """
        self.load_jobs()
        logging.info("Scheduler started.")
        try:
            while True:
                schedule.run_pending()
                time.sleep(1)
        except KeyboardInterrupt:
            self.shutdown()

    def shutdown(self) -> None:
        """Shut down the scheduler and save the current job state."""
        logging.info("Scheduler shutting down...")
        self.save_jobs()
        schedule.clear()
        logging.info("Scheduler has been shut down and state saved.")

    def load_jobs(self) -> None:
        """
        Load and schedule jobs from the state file.

        Clears existing jobs before loading to prevent duplication on restart.
        """
        self.jobs.clear()
        schedule.clear()

        if not os.path.exists(self.state_file):
            logging.warning(
                f"State file '{self.state_file}' not found. "
                "Starting with no jobs."
            )
            return

        try:
            with open(self.state_file, 'r') as f:
                job_definitions = json.load(f)

            if not isinstance(job_definitions, list):
                logging.error(
                    f"Corrupted state file"
                    f" '{self.state_file}': Expected a list."
                )
                return

            logging.info(f"Loading jobs from '{self.state_file}'...")
            for job_def in job_definitions:
                try:
                    self.add_job(
                        job_id=job_def.get("id"),
                        interval=job_def.get("interval"),
                        unit=job_def.get("unit"),
                        action=job_def.get("action"),
                        args=job_def.get("args", [])
                    )
                except Exception as e:
                    logging.error(
                        f"Failed to load job from definition {job_def}: {e}"
                    )

        except json.JSONDecodeError:
            logging.error(
                f"Invalid JSON in state file '{self.state_file}'. "
                "Starting with no jobs."
            )
        except IOError as e:
            logging.error(
                f"Failed to read state file '{self.state_file}': {e}"
            )

    def save_jobs(self) -> None:
        """Save the current list of job definitions to the state file."""
        job_definitions = list(self.jobs.values())
        try:
            with open(self.state_file, 'w') as f:
                json.dump(job_definitions, f, indent=2)
            logging.info(
                f"Saved {len(job_definitions)} jobs to '{self.state_file}'."
            )
        except IOError as e:
            logging.error(
                f"Failed to write to state file '{self.state_file}': {e}"
            )
        except TypeError as e:
            logging.error(f"Error serializing job definitions: {e}")


if __name__ == "__main__":
    STATE_FILE = "jobs_state.json"

    initial_jobs = [
        {
            "id": "job1",
            "interval": 5,
            "unit": "seconds",
            "action": "print_message",
            "args": ["Job 1 executed"]
        },
        {
            "id": "job2",
            "interval": 1,
            "unit": "minutes",
            "action": "print_message",
            "args": ["Job 2 running every minute"]
        },
        {
            "id": "job3_invalid",
            "interval": -10,
            "unit": "seconds",
            "action": "print_message",
            "args": ["This job should be ignored"]
        },
        {
            "id": "job4_unknown_action",
            "interval": 10,
            "unit": "seconds",
            "action": "unknown_function",
            "args": []
        }
    ]

    if not os.path.exists(STATE_FILE):
        logging.info(
            f"Creating initial job state file '{STATE_FILE}' for first run."
        )
        try:
            with open(STATE_FILE, 'w') as f:
                json.dump(initial_jobs, f, indent=2)
        except IOError as e:
            logging.error(f"Could not create initial state file: {e}")

    scheduler = JobScheduler(state_file=STATE_FILE)
    scheduler.start()


In [ ]:
# tests


"""Unit tests for JobScheduler class covering all functionalities.

Import modules and define tests for JobScheduler, covering all edge cases.
"""

import json
import os
import tempfile
import unittest
from unittest.mock import patch
import schedule

from main import JobScheduler


class TestJobScheduler(unittest.TestCase):
    """Test JobScheduler class functionality."""

    def setUp(self):
        """Set up test environment before each test."""
        self.test_state_file = tempfile.mktemp(suffix='.json')
        self.scheduler = JobScheduler(self.test_state_file)
        schedule.clear()  # Clear any existing scheduled jobs.

    def tearDown(self):
        """Clean up test environment after each test."""
        schedule.clear()
        if os.path.exists(self.test_state_file):
            os.unlink(self.test_state_file)

    def test_init_creates_scheduler_with_state_file(self):
        """Test JobScheduler initialization with state file."""
        scheduler = JobScheduler("test_file.json")
        self.assertEqual(scheduler.state_file, "test_file.json")
        # Only test initialization, not internal structure.

    def test_add_job_valid_parameters_success(self):
        """Test adding a valid job successfully."""
        self.scheduler.add_job(
            "job1", 5, "seconds", "print_message", ["Hello"])

        # Verify job was added by checking if it can be saved/loaded.
        self.scheduler.save_jobs()
        self.assertTrue(os.path.exists(self.test_state_file))

        # Load and verify the job exists in the saved state.
        with open(self.test_state_file, 'r') as f:
            saved_data = json.load(f)
        self.assertGreater(len(saved_data), 0)
        job_found = any(job.get("id") == "job1" for job in saved_data)
        self.assertTrue(job_found)

    def test_add_job_duplicate_id_warns_and_skips(self):
        """Test adding job with duplicate ID warns and skips addition."""
        self.scheduler.add_job(
            "job1", 5, "seconds", "print_message", ["First"])

        # Save first job state.
        self.scheduler.save_jobs()
        with open(self.test_state_file, 'r') as f:
            first_save = json.load(f)

        with patch('logging.warning') as mock_warning:
            self.scheduler.add_job(
                "job1", 10, "minutes", "print_message", ["Second"])
            # Should warn about duplicate ID.

        # Save again and compare - should be unchanged.
        self.scheduler.save_jobs()
        with open(self.test_state_file, 'r') as f:
            second_save = json.load(f)

        # First job should remain unchanged (same data structure).
        self.assertEqual(len(first_save), len(second_save))
        if len(first_save) > 0:
            self.assertEqual(first_save[0]["interval"], 5)
            self.assertEqual(first_save[0]["unit"], "seconds")

    def test_add_job_validates_parameters_correctly(self):
        """Test add_job validates parameters according to constraints."""
        # Test that invalid parameters do not cause crashes.
        invalid_jobs = [
            ("invalid-job", 5, "seconds", "print_message", []),
            ("job2", -5, "seconds", "print_message", []),
            ("job3", 0, "seconds", "print_message", []),
            ("job4", 1000001, "seconds", "print_message", []),
            ("job5", 5, "invalid_unit", "print_message", []),
            ("a" * 65, 5, "seconds", "print_message", [])
        ]

        for job_args in invalid_jobs:
            try:
                self.scheduler.add_job(*job_args)
                self.assertTrue(True)
            except Exception as e:
                self.fail(
                    f"add_job should handle invalid params gracefully: {e}")

        # Test valid job works.
        self.scheduler.add_job(
            "validjob1", 5, "seconds", "print_message", ["Hello"])

        # Save and verify at least valid job handling works.
        self.scheduler.save_jobs()
        with open(self.test_state_file, 'r') as f:
            saved_data = json.load(f)
        self.assertIsInstance(saved_data, list)

    def test_save_jobs_creates_correct_json_file(self):
        """Test save_jobs creates correct JSON file format."""
        self.scheduler.add_job(
            "job1", 5, "seconds", "print_message", ["Test message"])

        self.scheduler.save_jobs()

        self.assertTrue(os.path.exists(self.test_state_file))
        with open(self.test_state_file, 'r') as f:
            saved_data = json.load(f)

        self.assertIsInstance(saved_data, list)
        self.assertEqual(len(saved_data), 1)
        self.assertEqual(saved_data[0]["id"], "job1")

    def test_load_jobs_missing_file_starts_with_no_jobs(self):
        """Test load_jobs handles missing state file gracefully."""
        non_existent_file = "non_existent_file.json"
        scheduler = JobScheduler(non_existent_file)

        # Should not crash when loading non-existent file.
        try:
            scheduler.load_jobs()
            self.assertTrue(True)
        except Exception as e:
            self.fail(f"load_jobs should handle missing file: {e}")

        # After loading, there should be no jobs to save.
        scheduler.save_jobs()
        if os.path.exists(non_existent_file):
            with open(non_existent_file, 'r') as f:
                data = json.load(f)
            self.assertEqual(len(data), 0)
            os.unlink(non_existent_file)

    def test_load_jobs_invalid_json_starts_with_no_jobs(self):
        """Test load_jobs handles corrupted JSON gracefully."""
        with open(self.test_state_file, 'w') as f:
            f.write("invalid json content")

        # Should not crash with invalid JSON.
        try:
            self.scheduler.load_jobs()
            self.assertTrue(True)
        except Exception as e:
            self.fail(f"load_jobs should handle invalid JSON: {e}")

        # Should result in no jobs being loaded.
        self.scheduler.save_jobs()
        with open(self.test_state_file, 'r') as f:
            data = json.load(f)
        self.assertEqual(len(data), 0)

    def test_load_jobs_non_list_format_starts_with_no_jobs(self):
        """Test load_jobs handles non-list JSON format gracefully."""
        with open(self.test_state_file, 'w') as f:
            json.dump({"not": "a list"}, f)

        # Should not crash with wrong JSON format.
        try:
            self.scheduler.load_jobs()
            self.assertTrue(True)
        except Exception as e:
            self.fail(f"load_jobs should handle wrong format: {e}")

        # Should result in no jobs being loaded.
        self.scheduler.save_jobs()
        with open(self.test_state_file, 'r') as f:
            data = json.load(f)
        self.assertEqual(len(data), 0)

    def test_load_jobs_valid_file_loads_jobs_correctly(self):
        """Test load_jobs correctly loads valid job definitions."""
        test_jobs = [
            {
                "id": "job1",
                "interval": 5,
                "unit": "seconds",
                "action": "print_message",
                "args": ["Test 1"]
            },
            {
                "id": "job2",
                "interval": 10,
                "unit": "minutes",
                "action": "print_message",
                "args": ["Test 2"]
            }
        ]

        with open(self.test_state_file, 'w') as f:
            json.dump(test_jobs, f)

        self.scheduler.load_jobs()

        # Verify jobs loaded by saving and checking the saved state.
        self.scheduler.save_jobs()
        with open(self.test_state_file, 'r') as f:
            saved_data = json.load(f)

        self.assertGreaterEqual(len(saved_data), 1)
        job_ids = [job.get("id") for job in saved_data]
        self.assertIn("job1", job_ids)
        self.assertIn("job2", job_ids)

    def test_load_jobs_handles_invalid_job_definitions_gracefully(self):
        """Test load_jobs handles invalid job definitions gracefully."""
        test_jobs = [
            {
                "id": "valid_job",
                "interval": 5,
                "unit": "seconds",
                "action": "print_message",
                "args": ["Valid"]
            },
            {
                "id": "invalid_job",
                "interval": -5,
                "unit": "seconds",
                "action": "print_message",
                "args": ["Invalid"]
            }
        ]

        with open(self.test_state_file, 'w') as f:
            json.dump(test_jobs, f)

        # Test should not crash and should handle invalid jobs gracefully.
        try:
            self.scheduler.load_jobs()
            self.assertTrue(True)
        except Exception as e:
            self.fail(
                f"load_jobs should handle invalid jobs gracefully: {e}")

    def test_load_jobs_handles_missing_required_fields(self):
        """Test load_jobs handles jobs with missing required fields."""
        test_jobs = [
            {
                "id": "missing_interval",
                "unit": "seconds",
                "action": "print_message",
                "args": ["Test"]
            },
            {
                "interval": 5,
                "unit": "seconds",
                "action": "print_message",
                "args": ["Missing ID"]
            }
        ]

        with open(self.test_state_file, 'w') as f:
            json.dump(test_jobs, f)

        # Should handle gracefully without crashing.
        try:
            self.scheduler.load_jobs()
            self.assertTrue(True)
        except Exception as e:
            self.fail(f"Should handle missing fields gracefully: {e}")

    def test_add_job_with_missing_fields_fails_validation(self):
        """Test add_job rejects jobs with missing required fields."""
        # Test missing interval (None).
        self.scheduler.add_job(
            "job1", None, "seconds", "print_message", [])

        # Test missing unit (None).
        self.scheduler.add_job(
            "job2", 5, None, "print_message", [])

        # Test missing action (None).
        self.scheduler.add_job(
            "job3", 5, "seconds", None, [])

        # Verify that invalid jobs do not get saved.
        self.scheduler.save_jobs()
        with open(self.test_state_file, 'r') as f:
            saved_data = json.load(f)

        job_ids = [job.get("id") for job in saved_data]
        self.assertNotIn("job1", job_ids)
        self.assertNotIn("job2", job_ids)
        self.assertNotIn("job3", job_ids)

    @patch('time.sleep')
    @patch('schedule.run_pending')
    def test_start_calls_schedule_run_pending_continuously(
            self, mock_run, mock_sleep):
        """Test start method calls schedule.run_pending continuously."""
        # Simulate KeyboardInterrupt after a few iterations.
        mock_run.side_effect = [None, None, KeyboardInterrupt()]

        with patch.object(self.scheduler, 'shutdown') as mock_shutdown:
            self.scheduler.start()
            mock_shutdown.assert_called_once()

        self.assertEqual(mock_run.call_count, 3)
        self.assertEqual(mock_sleep.call_count, 2)

    def test_shutdown_clears_schedule_and_saves_jobs(self):
        """Test shutdown method clears schedule and saves jobs."""
        self.scheduler.add_job(
            "job1", 5, "seconds", "print_message", ["Test"])

        with patch.object(self.scheduler, 'save_jobs') as mock_save:
            self.scheduler.shutdown()
            mock_save.assert_called_once()

        self.assertEqual(len(schedule.jobs), 0)

    def test_scheduler_handles_job_execution_simulation(self):
        """Test scheduler can handle job execution simulation."""
        # Add a valid job.
        self.scheduler.add_job(
            "test_job", 1, "seconds", "print_message", ["Test execution"])

        # The key test is that schedule.run_pending() does not crash.
        with patch('logging.info'):
            try:
                schedule.run_pending()
                self.assertTrue(True)
            except Exception as e:
                self.fail(
                    f"schedule.run_pending() should not crash: {e}")

    def test_multiple_jobs_can_be_scheduled_simultaneously(self):
        """Test multiple jobs can be scheduled without conflict."""
        jobs_data = [
            ("job1", 5, "seconds", ["Message 1"]),
            ("job2", 10, "minutes", ["Message 2"]),
            ("job3", 2, "hours", ["Message 3"]),
            ("job4", 1, "days", ["Message 4"])
        ]

        for job_id, interval, unit, args in jobs_data:
            self.scheduler.add_job(
                job_id, interval, unit, "print_message", args)

        # Test that multiple add_job calls do not interfere with each other.
        self.scheduler.save_jobs()
        with open(self.test_state_file, 'r') as f:
            saved_data = json.load(f)

        self.assertIsInstance(saved_data, list)

    def test_persistence_round_trip_maintains_job_data(self):
        """Test save and load cycle maintains job data integrity."""
        original_jobs = [
            ("job1", 5, "seconds", ["Test 1"]),
            ("job2", 30, "minutes", ["Test 2", "Multiple", "Args"])
        ]

        for job_id, interval, unit, args in original_jobs:
            self.scheduler.add_job(
                job_id, interval, unit, "print_message", args)
        self.scheduler.save_jobs()

        new_scheduler = JobScheduler(self.test_state_file)
        new_scheduler.load_jobs()

        new_scheduler.save_jobs()
        with open(self.test_state_file, 'r') as f:
            loaded_data = json.load(f)

        self.assertGreaterEqual(len(loaded_data), 1)
        job_ids = [job.get("id") for job in loaded_data]
        self.assertIn("job1", job_ids)
        self.assertIn("job2", job_ids)

    def test_scheduler_handles_up_to_100_jobs(self):
        """Test scheduler can handle up to 100 jobs as per requirements."""
        # Add 10 jobs (representative test).
        for i in range(10):
            job_id = f"job{i:03d}"
            self.scheduler.add_job(
                job_id, i + 1, "seconds", "print_message", [f"Message {i}"])

        self.scheduler.save_jobs()
        with open(self.test_state_file, 'r') as f:
            data = json.load(f)

        self.assertIsInstance(data, list)

        new_scheduler = JobScheduler(self.test_state_file)
        try:
            new_scheduler.load_jobs()
            self.assertTrue(True)
        except Exception as e:
            self.fail(f"Should handle loading multiple jobs: {e}")

    def test_job_state_file_named_correctly(self):
        """Test that jobs are saved to 'jobs_state.json' file."""
        scheduler = JobScheduler("jobs_state.json")
        scheduler.add_job(
            "test", 5, "seconds", "print_message", ["Test"])
        scheduler.save_jobs()

        self.assertTrue(os.path.exists("jobs_state.json"))

        if os.path.exists("jobs_state.json"):
            os.unlink("jobs_state.json")

    def test_all_time_units_supported(self):
        """Test all required time units are supported."""
        valid_units = ["seconds", "minutes", "hours", "days"]

        for unit in valid_units:
            job_id = f"test_{unit}"
            try:
                self.scheduler.add_job(
                    job_id, 1, unit, "print_message", [f"Test {unit}"])
                self.assertTrue(True)
            except Exception as e:
                self.fail(f"Should support time unit '{unit}': {e}")

    def test_json_serializable_args_only(self):
        """Test that only JSON-serializable arguments are accepted."""
        valid_args = [
            ["string", 123, 45.67, True, False, None],
            [{"key": "value"}, [1, 2, 3]]
        ]

        for args in valid_args:
            try:
                self.scheduler.add_job(
                    f"valid_{hash(str(args))}", 5, "seconds",
                    "print_message", args)
                self.assertTrue(True)
            except Exception as e:
                self.fail(f"Should accept JSON-serializable args: {e}")


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: <Add the URL here>

#### Model code:

```python
# code generated by the model
import schedule
import time
import json
import os
from datetime import datetime
from typing import Callable, Dict, Any

def print_message(message: str) -> None:
    print(f"Message: {message}")

# Define any additional actions here
# def another_action(...): ...

class JobScheduler:
    def __init__(self, state_file: str = "jobs_state.json") -> None:
        self.state_file = state_file
        self.jobs: Dict[str, schedule.Job] = {}
        self.actions: Dict[str, Callable] = {
            "print_message": print_message
        }
        self.load_jobs()

    def add_job(self, job_id: str, interval: int, unit: str, action: str, args: list) -> None:
        if job_id in self.jobs:
            print(f"Warning: Job ID {job_id} is already scheduled. Skipping.")
            return

        action_func = self.actions.get(action)
        if not action_func:
            print(f"Error: Unknown action '{action}' for Job ID {job_id}.")
            return

        # Determine the schedule method based on unit
        schedule_method = getattr(schedule.every(interval), unit, None)
        if not schedule_method:
            print(f"Error: Unsupported time unit '{unit}' for Job ID {job_id}.")
            return

        # Define the job function
        def job_func():
            timestamp = datetime.now().isoformat()
            print(f"[{timestamp}] Executed job {job_id}: {action} {args}")
            action_func(*args)

        # Schedule the job
        job = schedule_method.do(job_func)
        self.jobs[job_id] = job

    def start(self) -> None:
        try:
            while True:
                schedule.run_pending()
                time.sleep(1)
        except KeyboardInterrupt:
            print("Shutdown signal received. Saving jobs state and exiting.")
            self.shutdown()

    def shutdown(self) -> None:
        self.save_jobs()
        print("Scheduler stopped.")

    def load_jobs(self) -> None:
        if not os.path.exists(self.state_file):
            return
        try:
            with open(self.state_file, 'r') as f:
                jobs_data = json.load(f)
                for job in jobs_data:
                    self.add_job(job["id"], job["interval"], job["unit"], job["action"], job["args"])
        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error loading jobs from {self.state_file}: {e}")

    def save_jobs(self) -> None:
        jobs_data = [{
            "id": job_id,
            "interval": job.interval.value,  # Note: This is illustrative, actual extraction might differ # <Issue> Jobs are not saved in a reconstructible format.
            "unit": job.interval.unit, # <Issue> Restarted jobs cannot be reliably restored.
            "action": job.job_func.__name__,
            "args": job_func_args  # This assumes args can be re-extracted
            # <Issue> job_func_args is undefined, cannot serialize arguments
        } for job_id, job in self.jobs.items()]
        with open(self.state_file, 'w') as f:
            json.dump(jobs_data, f, indent=4)

if __name__ == "__main__":
    scheduler = JobScheduler()

    # Example setup - typically jobs would be loaded from a file at start
    sample_jobs = [
        {
            "id": "job1",
            "interval": 5,
            "unit": "seconds",
            "action": "print_message",
            "args": ["Job 1 executed"]
        },
        {
            "id": "job2",
            "interval": 1,
            "unit": "minutes",
            "action": "print_message",
            "args": ["Job 2 running every minute"]
        }
    ]
    
    for job in sample_jobs:
        scheduler.add_job(job["id"], job["interval"], job["unit"], job["action"], job["args"])
    
    scheduler.start()
```